### fake_news_dataのEDA

In [1]:
import os
import pickle
import pandas as pd
import re
import datetime
from sklearn.preprocessing import LabelEncoder

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 5000)

In [2]:
def load(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

def dump(value, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'wb') as f:
        pickle.dump(value, f)

In [34]:
scraping_data_path = '/Users/kanekotakafumi/github/fake_news_detection_research/data/scraping_data/'
dataset_path = '/Users/kanekotakafumi/github/fake_news_detection_research/data/dataset/'
fij_tweet_data = load(scraping_data_path+'fij_news_data/tweet_data.pkl')
infact_tweet_data = load(scraping_data_path+'infact_news_data/tweet_data.pkl')
nikkei_tweet_data = load(scraping_data_path+'nikkei_news_data/tweet_data.pkl')
infact_fake_tweet_text_list = load(scraping_data_path+'infact_news_data/fake_twitter_text_list.pkl')
fij_fake_tweet_text_list = load(scraping_data_path+'fij_news_data/fake_twitter_text_list.pkl')
fij_fake_tweet_text_dict = load(scraping_data_path+'fij_news_data/fake_twitter_text_dict.pkl')
infact_fake_tweet_text_dict = load(scraping_data_path+'infact_news_data/fake_twitter_text_dict.pkl')
nikkei_news_title_list = load(scraping_data_path+'nikkei_news_data/nikkei_news_title_list.pkl')

In [37]:
fij_fake_tweet_text_dict

{'（うるま市で発生したクラスター）感染者の9割は（ワクチン）2回接種済みだった': ['2回接種済みだった'],
 '新型コロナウイルスのワクチン接種が変異ウイルスを生み出した': ['新型コロナウイルスのワクチン接種が変異ウイルスを生み出した'],
 '水でPCR検査「陽性」': ['水でPCR検査「陽性」'],
 '米国CDC『PCR検査は精度が低いので中止します』': ['米国CDC『PCR検査は精度が低いので中止します』'],
 'ワクチンは感染予防効果を期待できるものではない': ['ワクチンは感染予防効果を期待できるものではない'],
 '新型コロナのワクチンを打つと、自分の遺伝子が組み換わってしまう': ['新型コロナのワクチンを打つと、自分の遺伝子が組み換わってしまう'],
 'コロナワクチンの治験は終わっていない': ['コロナワクチンの治験は終わっていない'],
 'コロナワクチンでプラセボ（生理食塩水）を接種される可能性がある': ['コロナワクチンでプラセボを接種される可能性がある'],
 'ファイザー社が公表している研究計画書には、ワクチン接種者の息や汗など体液からスパイクタンパク質が放出され、妊娠中や妊娠計画中、授乳中の女性や乳児に有害事象を引き起こす可能性が示されている': ['ファイザー社が公表している研究計画書には、ワクチン接種者の息や汗など体液からスパイクタンパク質が放出され、妊娠中や妊娠計画中、授乳中の女性や乳児に有害事象を引き起こす可能性が示されている'],
 '日本の水道水を浴びたコロナ、一瞬で死滅、感染拡大地域はミネラルウォーターが原因か': ['日本の水道水を浴びたコロナ、一瞬で死滅、感染拡大地域はミネラルウォーターが原因か'],
 'コロナワクチンが流産や不妊につながる': ['コロナワクチンが流産や不妊につながる'],
 '米ファイザー社の公式文書に、ワクチン接種者が感染源になることを示す記載がある': ['米ファイザー社の公式文書に、ワクチン接種者が感染源になることを示す記載がある'],
 'ワクチンの中身はマイクロチップ（洗脳）・放射性同位体（体内被ばく）・組み換え遺伝子配列（ミュータント化）・塩化カリウム（安楽死用の劇薬）・DHMO（Dihydrogen Monoxide）（爆発）': ['ワクチンの中身はマ

In [38]:
infact_fake_tweet_text_dict

{'ドイツのクリニック所長がコロナワクチンの危険性を訴え自殺': ['ドイツのクリニック所長がコロナワクチンの危険性を訴え自殺'],
 '米CDC、PCR検査は精度が低いので中止': ['このウルトラ大ニュースを未だに日本のマスコミはこれを大きく報道しない',
  '米国CDC『PCR検査は精度が低いので中止します』',
  '米CDC、PCR検査は精度が低いので中止'],
 'スウェーデン首相がコロナ禍初期に感染拡大を止められなかった責任を取って辞任': ['スウェーデン首相がコロナ禍初期に感染拡大を止められなかった責任を取って辞任'],
 '日本の水道水を浴びたコロナ、一瞬で死滅': ['日本の水道水を浴びたコロナ、一瞬で死滅'],
 'マクドナルド、バドワイザーが五輪のスポンサーを降りた。多くのスポンサー企業が東京五輪に冷めている': ['に一番冷めているらしいです',
  'マクドナルド、バドワイザーが五輪のスポンサーを降りた。多くのスポンサー企業が東京五輪に冷めている',
  'マクドナルドやバドワイザーは五輪のスポンサーを降りたんですね。多くのスポンサー企業が'],
 '大阪の新規感染者1260人のうち日本人は1人も確認されていない': ['大阪の新規感染者1260人のうち日本人は1人も確認されていない'],
 'ワクチンの中のナノテクノロジーロボット': ['ワクチンの中のナノテクノロジーロボット'],
 '「日本初の死亡者も…」': ['「日本初の死亡者も…」'],
 '罰則の創設を求める緊急提言を頂いている。': ['罰則の創設を求める緊急提言を頂いている。', '全国知事会からも'],
 '史上最大の詐欺の１つ」': ['ニューヨーク・タイムズ紙「PCR検査陽性の9割は誤診だった」',
  '「PCR検査がコロナ感染を検出する為のゴールドスタンダードだと騙した詐欺は',
  '史上最大の詐欺の１つ」'],
 '男性の3倍の人が命を断っている': ['女性186人。女性に何があったのか',
  '総数は246人だけど',
  '男性60人',
  '男性の3倍の人が命を断っている'],
 'そして命を救う治療を優先している。': ['そして命を救う治療を優先している。',
  'できるだけ早めに気管切開をして',
  '治療的な観点でいくと',
 

In [4]:
fij_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34563 entries, 0 to 0
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   text                 34563 non-null  object
 1   id                   34563 non-null  object
 2   conversation_id      34563 non-null  object
 3   created_at           34563 non-null  object
 4   in_reply_to_user_id  978 non-null    object
 5   author_id            34563 non-null  object
 6   kw                   34563 non-null  object
dtypes: object(7)
memory usage: 2.1+ MB


In [5]:
infact_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13324 entries, 0 to 499
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            13324 non-null  object
 1   id                   13324 non-null  object
 2   created_at           13324 non-null  object
 3   conversation_id      13324 non-null  object
 4   text                 13324 non-null  object
 5   kw                   13324 non-null  object
 6   in_reply_to_user_id  511 non-null    object
dtypes: object(7)
memory usage: 832.8+ KB


In [6]:
nikkei_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107486 entries, 0 to 0
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id                   107486 non-null  object
 1   created_at           107486 non-null  object
 2   author_id            107486 non-null  object
 3   conversation_id      107486 non-null  object
 4   text                 107486 non-null  object
 5   kw                   107486 non-null  object
 6   in_reply_to_user_id  1568 non-null    object
dtypes: object(7)
memory usage: 6.6+ MB


In [7]:
fij_tweet_data.nunique()

text                    7280
id                     33462
conversation_id        33402
created_at             32348
in_reply_to_user_id      884
author_id              25948
kw                       150
dtype: int64

<br>
<p>listを確認するとフェイクニュースと関係ない用語が含まれているので一部除外する必要あり。</p>

In [8]:
len(fij_fake_tweet_text_list)

183

In [9]:
len(nikkei_news_title_list)

1865

In [12]:
len(infact_fake_tweet_text_list)

41

In [13]:
print(len(fij_tweet_data))
fij_tweet_data = fij_tweet_data[~fij_tweet_data['kw'].isin(['日本', 'go', '2回接種済みだった', '機能停止した', '除染作業'])]
print(len(fij_tweet_data))

34563
32582


In [19]:
fij_tweet_data['kw'].unique()

array(['新型コロナウイルスのワクチン接種が変異ウイルスを生み出した', '水でPCR検査「陽性」',
       '米国CDC『PCR検査は精度が低いので中止します』', 'ワクチンは感染予防効果を期待できるものではない',
       'コロナワクチンの治験は終わっていない', '日本の水道水を浴びたコロナ、一瞬で死滅、感染拡大地域はミネラルウォーターが原因か',
       'コロナワクチンが流産や不妊につながる', '米ファイザー社の公式文書に、ワクチン接種者が感染源になることを示す記載がある',
       'ワクチンの中身はマイクロチップ', '「1回目の接種でまず一次的な免疫をつけていただく」',
       'ワクチンが卵巣に蓄積、不妊の原因に', 'ファイザー社の公式サイトに不妊症を引き起こす可能性があると記されている',
       '胎盤に影響を及ぼしたり、免疫反応に影響を及ぼすことで妊娠のプロセスを乱す', '無期限に不妊症を起こす危険性がある',
       '菅首相がうったワクチンは偽物', '日本初の死亡者',
       '日本の高校生320万人全員にワクチンをうったら、確率的には50人が死亡するか、それに相当する副作用に見舞われる',
       '筋肉に注射するタイプのワクチンが呼吸器感染症に効くというのは、合理的じゃない',
       'ファイザー製コロナワクチンを打った人にエボラの症状。全身から血が吹き出して死亡',
       'ワクチンは感染予防にはつながらず、あくまで重症化予防のために打つためのもの', '(大江戸線の集団感染)「洗面所の蛇口が原因か」',
       '現在の日本の新型コロナウイルスは', '1/10000000の死亡率',
       'ニューヨーク・タイムズ紙が「PCR検査陽性の9割は誤診だった」と報道した',
       'PCR法を開発してノーベル賞を取った人が「ウイルスの特定にはふさわしくない」と言った', 'マスクによる飛沫感染リスク低減率',
       'イソジンなどのうがい薬でコロナ感染予防', 'WHOが方向転換「感染者の隔離は不要」', 'PCR検査は風邪も検出',
       'このコロナウイルス、高温多湿と紫外線が大嫌い', '加湿器等に

In [24]:
infact_tweet_data['kw'].unique()

array(['ドイツのクリニック所長がコロナワクチンの危険性を訴え自殺', '米CDC、PCR検査は精度が低いので中止',
       '米国CDC『PCR検査は精度が低いので中止します』', 'このウルトラ大ニュースを未だに日本のマスコミはこれを大きく報道しない',
       'スウェーデン首相がコロナ禍初期に感染拡大を止められなかった責任を取って辞任', '日本の水道水を浴びたコロナ、一瞬で死滅',
       'マクドナルド、バドワイザーが五輪のスポンサーを降りた。多くのスポンサー企業が東京五輪に冷めている',
       'マクドナルドやバドワイザーは五輪のスポンサーを降りたんですね。多くのスポンサー企業が', 'に一番冷めているらしいです',
       '大阪の新規感染者1260人のうち日本人は1人も確認されていない', 'ワクチンの中のナノテクノロジーロボット',
       '「日本初の死亡者も…」', '全国知事会からも', '罰則の創設を求める緊急提言を頂いている。',
       '「PCR検査がコロナ感染を検出する為のゴールドスタンダードだと騙した詐欺は', '史上最大の詐欺の１つ」',
       '総数は246人だけど', '男性60人', '治療的な観点でいくと', '大阪の場合は',
       '死者をできるだけ減らしたいということで', 'できるだけ早めに気管切開をして', '人工呼吸器をつけて', 'PCR',
       '法を開発したノーベル賞を取った人がウイルスの特定にはふさわしくないと言っている', '保健所の削減は太田府政時代ですよ。',
       'WHOが急に方向転換', '新型コロナウィルス感染者の隔離は必要ない', '検疫も必要ない',
       'ソーシャルディスタンスも必要ない', '感染者からも感染しない', 'と言い出した。', '日本政府は'],
      dtype=object)

In [8]:
fij_tweet_data[fij_tweet_data.duplicated('id', keep=False)].head()

,text,id,conversation_id,created_at,in_reply_to_user_id,author_id,kw
0,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358559259827441665,1358559259827441665,2021-02-07T23:32:23.000Z,NaN,3163900800,現在の日本の新型コロナウイルスは
1,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358520429942501379,1358520429942501379,2021-02-07T20:58:05.000Z,NaN,594382119,現在の日本の新型コロナウイルスは
2,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358434939662868480,1358434939662868480,2021-02-07T15:18:22.000Z,NaN,1230546793043116032,現在の日本の新型コロナウイルスは
3,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358433072576745480,1358433072576745480,2021-02-07T15:10:57.000Z,NaN,1311335247720542210,現在の日本の新型コロナウイルスは
4,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358311231560380419,1358311231560380419,2021-02-07T07:06:48.000Z,NaN,341556219,現在の日本の新型コロナウイルスは


In [14]:
nikkei_tweet_data[nikkei_tweet_data.duplicated('id', keep=False)].head()

,id,created_at,author_id,conversation_id,text,kw,in_reply_to_user_id
137,1384026312881315842,2021-04-19T06:09:21.000Z,126193109,1384025027599736833,大阪知事、緊急事態宣言要請へ 「感染は危機的状況」 https://t.co/8J4xWEC...,大阪知事、緊急事態宣言要請へ 「感染は危機的状況」,126193109
27,1433463083842752514,2021-09-02T16:13:26.000Z,84771675,1433463076183949317,当初は日経がスクープ。\n\nコロナ病床実態調査へ 政府、補助金受け消極的な病院も: 日本経...,コロナ病床実態調査へ 政府、補助金受け消極的な病院も,84771675
224,1433463085776338944,2021-09-02T16:13:26.000Z,84771675,1433463076183949317,新型コロナ: 厚労省と東京都、医療機関にコロナ患者受け入れ要請: 日本経済新聞 https:...,「名ばかり」のコロナ病床確保 東京、利用7割で頭打ち,84771675
43,1458459174765928452,2021-11-10T15:38:58.000Z,703726254994776065,1421775620686503947,『入国規制の緩和は踏み込みが足りない』\nhttps://t.co/JZquLk0H1A\n...,入国緩和に手続きの壁,703726254994776065
172,1458190068489719808,2021-11-09T21:49:38.000Z,1374334200254328836,1458190068489719808,2021年11月10日（水）\n・TSMC 日本工場に8000億円\n・自公 10万円支給 ...,入国緩和に手続きの壁,NaN


In [16]:
infact_tweet_data[infact_tweet_data.duplicated('id', keep=False)].head()

,author_id,id,created_at,conversation_id,text,kw,in_reply_to_user_id
0,2470599156,1435143112179916804,2021-09-07T07:29:16.000Z,1435143112179916804,RT @CYXuAxfGlfFzZCT: #PCR検査 \n米国CDC『PCR検査は精度が低...,米国CDC『PCR検査は精度が低いので中止します』,NaN
7,1060537556045950976,1427840162520657925,2021-08-18T03:49:57.000Z,1427840162520657925,RT @CYXuAxfGlfFzZCT: #PCR検査 \n米国CDC『PCR検査は精度が低...,米国CDC『PCR検査は精度が低いので中止します』,NaN
34,752818421285330944,1426158293257396235,2021-08-13T12:26:48.000Z,1426158293257396235,RT @CYXuAxfGlfFzZCT: #PCR検査 \n米国CDC『PCR検査は精度が低...,米国CDC『PCR検査は精度が低いので中止します』,NaN
35,4830547578,1426156875184177152,2021-08-13T12:21:10.000Z,1426156875184177152,RT @CYXuAxfGlfFzZCT: #PCR検査 \n米国CDC『PCR検査は精度が低...,米国CDC『PCR検査は精度が低いので中止します』,NaN
36,434046429,1426154786211078146,2021-08-13T12:12:52.000Z,1426154786211078146,RT @CYXuAxfGlfFzZCT: #PCR検査 \n米国CDC『PCR検査は精度が低...,米国CDC『PCR検査は精度が低いので中止します』,NaN


<p>複数のKWで同じtweetが抽出されているケースがある。</p>

In [9]:
fij_tweet_data[fij_tweet_data['id']=='1358559259827441665']

,text,id,conversation_id,created_at,in_reply_to_user_id,author_id,kw
0,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358559259827441665,1358559259827441665,2021-02-07T23:32:23.000Z,NaN,3163900800,現在の日本の新型コロナウイルスは
0,RT @katsuyatakasu: 現在の日本の新型コロナウイルスは\n1/1000000...,1358559259827441665,1358559259827441665,2021-02-07T23:32:23.000Z,NaN,3163900800,1/10000000の死亡率


In [10]:
fij_tweet_data = fij_tweet_data[~fij_tweet_data.duplicated('id')]
nikkei_tweet_data = nikkei_tweet_data[~nikkei_tweet_data.duplicated('id')]

In [11]:
fij_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31481 entries, 0 to 490
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   text                 31481 non-null  object
 1   id                   31481 non-null  object
 2   conversation_id      31481 non-null  object
 3   created_at           31481 non-null  object
 4   in_reply_to_user_id  777 non-null    object
 5   author_id            31481 non-null  object
 6   kw                   31481 non-null  object
dtypes: object(7)
memory usage: 1.9+ MB


In [12]:
def original_kw(value):
    return [k for k, v in fij_fake_tweet_text_dict.items() if value in v][0]

fij_tweet_data['event'] = fij_tweet_data['kw'].map(original_kw)

print(fij_tweet_data['event'].nunique())

le = LabelEncoder()
le.fit(fij_tweet_data['event'])

fij_tweet_data['event_id'] = le.transform(fij_tweet_data['event'])

fij_tweet_data['created_at'] = pd.to_datetime(fij_tweet_data['created_at'])

def timestamps_mk(df):
    for key in fij_tweet_data['event_id'].unique().tolist():
        base_time = fij_tweet_data[fij_tweet_data['event_id']==key].sort_values(['id'])[:1]['created_at'].tolist()[0]
        fij_tweet_data.loc[fij_tweet_data['event_id']==key, 'base_time'] = base_time
    
    fij_tweet_data['base_time'] = pd.to_datetime(fij_tweet_data['base_time'])
    
    return fij_tweet_data['created_at'] - fij_tweet_data['base_time']

fij_tweet_data['timestamps'] = timestamps_mk(fij_tweet_data)
fij_tweet_data['timestamps'] = fij_tweet_data['timestamps'].map(lambda x: x.total_seconds())

129


In [13]:
fij_tweet_data.head()

,text,id,conversation_id,created_at,in_reply_to_user_id,author_id,kw,event,event_id,base_time,timestamps
0,「『新型コロナウイルスのワクチン接種が変異ウイルスを生み出した』とする言説が拡散されている。...,1438799778515611652,1438799778515611652,2021-09-17 09:39:33+00:00,NaN,1214824720387858433,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,1211945.0
1,RT @BFJMedical: 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型...,1434630041430753281,1434630041430753281,2021-09-05 21:30:30+00:00,NaN,725833451455160320,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,217802.0
2,RT @BFJMedical: 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型...,1434508337480015874,1434508337480015874,2021-09-05 13:26:54+00:00,NaN,89932506,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,188786.0
3,「新型コロナウイルスのワクチン接種が変異ウイルスを生み出した」とする言説が拡散されている。ノ...,1434399241418412034,1418499748906946561,2021-09-05 06:13:23+00:00,848415134628237315,848415134628237315,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,162775.0
4,RT @BFJMedical: 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型...,1434369971807682571,1434369971807682571,2021-09-05 04:17:05+00:00,NaN,1384783568,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,155797.0


In [14]:
def text_extraction(text):
    text = re.sub(r'@\w*', '', text)
    text = re.sub(r'https:.*', '', text)
    
    return text

In [15]:
fij_tweet_data['text'] = fij_tweet_data['text'].map(text_extraction)

In [16]:
fij_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31481 entries, 0 to 490
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   text                 31481 non-null  object             
 1   id                   31481 non-null  object             
 2   conversation_id      31481 non-null  object             
 3   created_at           31481 non-null  datetime64[ns, UTC]
 4   in_reply_to_user_id  777 non-null    object             
 5   author_id            31481 non-null  object             
 6   kw                   31481 non-null  object             
 7   event                31481 non-null  object             
 8   event_id             31481 non-null  int64              
 9   base_time            31481 non-null  datetime64[ns, UTC]
 10  timestamps           31481 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(1), int64(1), object(7)
memory usage: 2.9+ MB


In [17]:
datset_dict = {}
for key in fij_tweet_data['event_id'].unique().tolist():
    messages_dict = {}
    messages_dict['id'] = fij_tweet_data[fij_tweet_data['event_id']==key].sort_values(['id'])['id'].tolist()
    messages_dict['uid'] = fij_tweet_data[fij_tweet_data['event_id']==key].sort_values(['id'])['author_id'].tolist()
    messages_dict['timestamps'] = fij_tweet_data[fij_tweet_data['event_id']==key].sort_values(['id'])['timestamps'].tolist()
    messages_dict['text'] = fij_tweet_data[fij_tweet_data['event_id']==key].sort_values(['id'])['text'].tolist()
    messages_dict['label'] = 1
    datset_dict[str(key)] = messages_dict

In [18]:
def word_segmentation(text:str, replace_dict:dict, exclude_task_list:list, a):
    
    token_list = []
    for token in a.analyze(text):
        token = token.translate(str.maketrans(replace_dict)) # 邪魔な文字を除く
        if token not in exclude_task_list and not token.isdecimal(): # 意味ないワードを除く
            token_list.append(token)
            token_list = [s for s in token_list if not s.startswith('#')]
            
    token_str = ' '.join(token_list)
    
    return token_str

In [19]:
replace_dict = {'[':'', ']':'', '/':'', '+':'', '(':'', ')':'', '等':'', ',':'', '.':'', '<':'', '>':'', '-':'', '?':'', ':':'', '|':''}
exclude_task_list = ['new', 'rt', '籏智広太', 'インファクト', 'ファクトチェック', 'factcheck', 'infact', 'こび', 'ナビ']

# analyzerモジュールで形態素分析
char_filters = [UnicodeNormalizeCharFilter()]

tokenizer = Tokenizer()

token_filters = [CompoundNounFilter(),
                 POSStopFilter(['記号']),
                 LowerCaseFilter(),
                 ExtractAttributeFilter('surface')]

a = Analyzer(char_filters=char_filters, tokenizer=tokenizer ,token_filters=token_filters)

for key in datset_dict.keys():
    text_word_segmentation_list = []
    for text in datset_dict[key]['text']:
        text_word_segmentation_list.append(word_segmentation(text, replace_dict, exclude_task_list, a))
    datset_dict[key]['text'] = text_word_segmentation_list

In [20]:
dump(datset_dict, dataset_path+'datset_dict.pkl')

In [39]:
datset_dict

NameError: name 'datset_dict' is not defined

In [21]:
fij_tweet_data_no_rt = fij_tweet_data[~fij_tweet_data['text'].str.contains('#')&(fij_tweet_data['id']==fij_tweet_data['conversation_id'])]
fij_tweet_data_no_rt.head(100)

,text,id,conversation_id,created_at,in_reply_to_user_id,author_id,kw,event,event_id,base_time,timestamps
0,「『新型コロナウイルスのワクチン接種が変異ウイルスを生み出した』とする言説が拡散されている。...,1438799778515611652,1438799778515611652,2021-09-17 09:39:33+00:00,NaN,1214824720387858433,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,1211945.0
1,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434630041430753281,1434630041430753281,2021-09-05 21:30:30+00:00,NaN,725833451455160320,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,217802.0
2,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434508337480015874,1434508337480015874,2021-09-05 13:26:54+00:00,NaN,89932506,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,188786.0
4,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434369971807682571,1434369971807682571,2021-09-05 04:17:05+00:00,NaN,1384783568,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,155797.0
5,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434252090721792002,1434252090721792002,2021-09-04 20:28:40+00:00,NaN,1055463594928726016,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,127692.0
6,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434232508112068610,1434232508112068610,2021-09-04 19:10:51+00:00,NaN,1261183915370459136,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,123023.0
7,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434186411809275909,1434186411809275909,2021-09-04 16:07:41+00:00,NaN,91602669,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,112033.0
8,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434171205569835016,1434171205569835016,2021-09-04 15:07:15+00:00,NaN,168032728,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,108407.0
10,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434167402191675393,1434167402191675393,2021-09-04 14:52:08+00:00,NaN,838979916,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,107500.0
11,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434147090611900417,1434147090611900417,2021-09-04 13:31:26+00:00,NaN,92180308,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,102658.0


In [22]:
fij_tweet_data_no_rt['text'].nunique()

3582

In [23]:
fij_tweet_data_no_rt['kw'].unique()

array(['新型コロナウイルスのワクチン接種が変異ウイルスを生み出した', '水でPCR検査「陽性」',
       '米国CDC『PCR検査は精度が低いので中止します』', 'ワクチンは感染予防効果を期待できるものではない',
       'コロナワクチンの治験は終わっていない', '日本の水道水を浴びたコロナ、一瞬で死滅、感染拡大地域はミネラルウォーターが原因か',
       'コロナワクチンが流産や不妊につながる', '米ファイザー社の公式文書に、ワクチン接種者が感染源になることを示す記載がある',
       'ワクチンの中身はマイクロチップ', '「1回目の接種でまず一次的な免疫をつけていただく」',
       'ワクチンが卵巣に蓄積、不妊の原因に', 'ファイザー社の公式サイトに不妊症を引き起こす可能性があると記されている',
       '無期限に不妊症を起こす危険性がある', '菅首相がうったワクチンは偽物', '日本初の死亡者',
       '日本の高校生320万人全員にワクチンをうったら、確率的には50人が死亡するか、それに相当する副作用に見舞われる',
       '筋肉に注射するタイプのワクチンが呼吸器感染症に効くというのは、合理的じゃない',
       'ワクチンは感染予防にはつながらず、あくまで重症化予防のために打つためのもの', '(大江戸線の集団感染)「洗面所の蛇口が原因か」',
       '現在の日本の新型コロナウイルスは', '1/10000000の死亡率',
       'ニューヨーク・タイムズ紙が「PCR検査陽性の9割は誤診だった」と報道した',
       'PCR法を開発してノーベル賞を取った人が「ウイルスの特定にはふさわしくない」と言った', 'マスクによる飛沫感染リスク低減率',
       'イソジンなどのうがい薬でコロナ感染予防', 'WHOが方向転換「感染者の隔離は不要」', 'PCR検査は風邪も検出',
       'このコロナウイルス、高温多湿と紫外線が大嫌い', '加湿器等に次亜塩素酸水を入れて噴霧することで空間除菌ができる',
       '感染対策に緑茶が有効', '「深く息を吸って、10秒我慢する」ことができれば新型コロナには感染していない',
      

In [24]:
fij_tweet_data[fij_tweet_data['kw']=='地元企業への利権']

,text,id,conversation_id,created_at,in_reply_to_user_id,author_id,kw,event,event_id,base_time,timestamps
0,また、総理大臣の立場だと地元に特化した立場も取りづらいですからね。\nマスクの件で「地元企...,1282882411416612864,1282720887754461185,2020-07-14 03:39:36+00:00,125633376,125633376,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,8873711.0
1,政府配布の布マスク、「受注3社」で差額300憶以上はどこに使った？もいいけど、肝腎なのは、...,1252686716781150209,1252674267830816769,2020-04-21 19:52:41+00:00,98891836,2340046106,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1674496.0
2,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1252010077243928577,1252010077243928577,2020-04-19 23:03:58+00:00,NaN,4676960132,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1513173.0
3,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251823219235074053,1251823219235074053,2020-04-19 10:41:27+00:00,NaN,310818431,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1468622.0
4,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251786391576436736,1251786391576436736,2020-04-19 08:15:07+00:00,NaN,2973960954,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1459842.0
5,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251731318984957954,1251731318984957954,2020-04-19 04:36:17+00:00,NaN,1106159828517310464,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1446712.0
6,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251702186293121024,1251702186293121024,2020-04-19 02:40:31+00:00,NaN,1857283026,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1439766.0
7,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251680854562050048,1251680854562050048,2020-04-19 01:15:45+00:00,NaN,247710561,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1434680.0
8,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251679602096799744,1251679602096799744,2020-04-19 01:10:46+00:00,NaN,1319996016,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1434381.0
9,RT : 「安倍晋三首相の地元である山口県の企業がマスク製造を受注するという誤解が広がり、...,1251673917116133376,1251673917116133376,2020-04-19 00:48:11+00:00,NaN,4165181053,地元企業への利権,（安倍首相の地元である）山口県の企業が布マスク製造を受注 地元企業への利権,115,2020-04-02 10:44:25+00:00,1433026.0


In [25]:
fij_tweet_data[fij_tweet_data['id']==fij_tweet_data['conversation_id']]

,text,id,conversation_id,created_at,in_reply_to_user_id,author_id,kw,event,event_id,base_time,timestamps
0,「『新型コロナウイルスのワクチン接種が変異ウイルスを生み出した』とする言説が拡散されている。...,1438799778515611652,1438799778515611652,2021-09-17 09:39:33+00:00,NaN,1214824720387858433,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,1211945.0
1,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434630041430753281,1434630041430753281,2021-09-05 21:30:30+00:00,NaN,725833451455160320,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,217802.0
2,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434508337480015874,1434508337480015874,2021-09-05 13:26:54+00:00,NaN,89932506,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,188786.0
4,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434369971807682571,1434369971807682571,2021-09-05 04:17:05+00:00,NaN,1384783568,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,155797.0
5,RT : 【New】ノーベル賞科学者のリュック・モンタニエ博士の「新型コロナウイルスのワクチ...,1434252090721792002,1434252090721792002,2021-09-04 20:28:40+00:00,NaN,1055463594928726016,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,新型コロナウイルスのワクチン接種が変異ウイルスを生み出した,68,2021-09-03 09:00:28+00:00,127692.0
...,...,...,...,...,...,...,...,...,...,...,...
486,RT : …,1305972362622320642,1305972362622320642,2020-09-15 20:50:49+00:00,NaN,736729596389793792,新型肺炎で中国人が押し寄せる危険性,中国で日本の健康保険に“悪乗り”する方法が拡散… 新型肺炎で中国人が押し寄せる危険性,42,2020-01-26 22:16:05+00:00,20126084.0
487,RT : …,1305884513910665216,1305884513910665216,2020-09-15 15:01:44+00:00,NaN,400038842,新型肺炎で中国人が押し寄せる危険性,中国で日本の健康保険に“悪乗り”する方法が拡散… 新型肺炎で中国人が押し寄せる危険性,42,2020-01-26 22:16:05+00:00,20105139.0
488,RT : …,1305824157251190790,1305824157251190790,2020-09-15 11:01:54+00:00,NaN,1223046367569838082,新型肺炎で中国人が押し寄せる危険性,中国で日本の健康保険に“悪乗り”する方法が拡散… 新型肺炎で中国人が押し寄せる危険性,42,2020-01-26 22:16:05+00:00,20090749.0
489,RT : …,1305811967601303553,1305811967601303553,2020-09-15 10:13:28+00:00,NaN,151538373,新型肺炎で中国人が押し寄せる危険性,中国で日本の健康保険に“悪乗り”する方法が拡散… 新型肺炎で中国人が押し寄せる危険性,42,2020-01-26 22:16:05+00:00,20087843.0


In [26]:
# fij_tweet_data[~fij_tweet_data['text'].str.contains('RT @')&~(fij_tweet_data['id']==fij_tweet_data['conversation_id'])]

In [27]:
# fij_tweet_data[~fij_tweet_data['text'].str.contains('RT @')]